Copy pandas dataframe to excel using openpyxl<br>
https://stackoverflow.com/questions/36657288/copy-pandas-dataframe-to-excel-using-openpyxl

How to delete one or more rows in excel using Openpyxl? Deleting all rows, Method 2<br>
https://www.geeksforgeeks.org/how-to-delete-one-or-more-rows-in-excel-using-openpyxl/

How to remove commas from ALL the column in pandas at once
https://stackoverflow.com/questions/56947333/how-to-remove-commas-from-all-the-column-in-pandas-at-once

Convert number strings with commas in pandas DataFrame to float<br>
https://stackoverflow.com/questions/22137723/convert-number-strings-with-commas-in-pandas-dataframe-to-float

Extracting date from a string in Python<br>
https://stackoverflow.com/questions/3276180/extracting-date-from-a-string-in-python

How to utilise the date_parser parameter of pandas.read_csv()<br>
https://stackoverflow.com/questions/60349071/how-to-utilise-the-date-parser-parameter-of-pandas-read-csv

Python date & time conversion CheatSheet<br>
https://dev.to/maikomiyazaki/python-date-time-conversion-cheatsheet-3m69#:~:text=To%20convert%20float%20of%20timestamp,argument%20and%20returns%20DateTime%20object.

A Complete Guide to Using Progress Bars in Python<br>
https://towardsdatascience.com/a-complete-guide-to-using-progress-bars-in-python-aa7f4130cda8

How do I list all files of a directory?<br>
https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory

Make the Most Out of your pandas.read_csv()<br>
https://medium.com/analytics-vidhya/make-the-most-out-of-your-pandas-read-csv-1531c71893b5

Converting column into proper timestamp using pandas read_csv<br>
https://stackoverflow.com/questions/42324119/converting-column-into-proper-timestamp-using-pandas-read-csv

In [1]:
import time # to calculate durations of runs
start_time = time.time()

import pandas as pd
from datetime import datetime
from pathlib import Path
import os
from tqdm import tqdm, notebook # notebook version of tqdm
#" ...any time you see a loop somewhere in your code in you can simply wrap it in either tdqm() or tqdm_notebook() in Jupyter" 

print(f"--- {round(start_time - time.time(),1)} seconds ---")

--- -1.2 seconds ---


In [2]:
# set variables and make dataframes out of the Eagle fund holdings and deltas .CSV workbooks
start_time = time.time()

# set path to the Derv.calc file
fH = r'C:\Users\hilton.netta\Downloads\Portfolio Analytics Report - New.csv' # the Eagle holdings spreadsheet
fD = r'C:\Users\hilton.netta\Downloads\Derivative Exposure.csv' # the Eagle delats spreadsheet
fDx = Path('//pim-cpt-fs/profile$/PProfile Operations/Legal/Legal and Compliance Framework/Hilton/W/Derv.xlsx') # Derv.xlsx
pthSave = r'C:\Users\hilton.netta\Documents\DervFiles' # path to were the files will be saved

# create dataframes of the Eagle holdings and deltas spreadsheets, using "thousands = ','" to deal with strings
wbD = pd.read_csv(fD, thousands=',') # holdings from Eagle

# convert string with commas to flaots and convert time strings in column AJ to "dd/mm/yyyy" format
z = lambda t: datetime.strptime(t) # creating a function to format the two important date columns P and AJ
wbH = pd.read_csv(fH, thousands=',') # holdings from Eagle
#https://stackoverflow.com/questions/60349071/how-to-utilise-the-date-parser-parameter-of-pandas-read-csv
print(type(wbH['i Position Effective Date'][0]),type(wbH['Maturity Date'][0]))

print(f"--- {round(start_time - time.time(),1)} seconds ---")

<class 'str'> <class 'str'>
--- -0.1 seconds ---


In [5]:
print(wbH['i Position Effective Date'][7], wbH['Maturity Date'][7])

27/05/2022 31/03/2046


In [4]:
print(datetime.strptime(wbH['i Position Effective Date'][0],"%d/%m/%Y"),datetime.strptime(wbH['Maturity Date'][0],"%d/%m/%Y"))

2022-05-23 00:00:00 2028-05-15 00:00:00


In [9]:
# set variables and make dataframes out of the Eagle fund holdings and deltas .CSV workbooks
start_time = time.time()

# set path to the Derv.calc file
fH = r'C:\Users\hilton.netta\Downloads\Portfolio Analytics Report - New.csv' # the Eagle holdings spreadsheet
fD = r'C:\Users\hilton.netta\Downloads\Derivative Exposure.csv' # the Eagle delats spreadsheet
fDx = Path('//pim-cpt-fs/profile$/PProfile Operations/Legal/Legal and Compliance Framework/Hilton/W/Derv.xlsx') # Derv.xlsx
pthSave = r'C:\Users\hilton.netta\Documents\DervFiles' # path to were the files will be saved

# create dataframes of the Eagle holdings and deltas spreadsheets, using "thousands = ','" to deal with strings
wbD = pd.read_csv(fD, thousands=',') # holdings from Eagle

# convert string with commas to flaots and convert time strings in column AJ to "dd/mm/yyyy" format
z = lambda t: datetime.strptime(t,"%d/%m/%Y") # creating a function to format the two important date columns P and AJ
wbH = pd.read_csv(fH, thousands=',', converters = {'i Position Effective Date':z,'Maturity Date':z}) # holdings from Eagle
#https://stackoverflow.com/questions/60349071/how-to-utilise-the-date-parser-parameter-of-pandas-read-csv

# get the report date and list of fund names
rptDate = wbH.iloc[0, 35].strftime("%#d%b%Y") # 1st row 36th column has the report date
fnames = wbH['Entity Name'].unique() # list of the fund names in the holdings file

print(f"--- {round(time.time() - start_time,1)} seconds ---")

ValueError: time data '' does not match format '%d/%m/%Y'

In [76]:
print(wbH['i Position Effective Date'][0],wbH['Maturity Date'][0])

18/05/2022 15/05/2028


In [6]:
# change the '% of Total Market Value" column (N) to the fund-specific % of NAV based on 'Sum of Market Value Income' column (M)
start_time = time.time()

navs = wbH.groupby('Entity ID')['Sum of Market Value Income'].sum() # (column N) this has type 'pandas.core.series.Series'
nav = navs.to_dict() # nav series changed to dictionary to make it lookupable

# recalc the '% of Total Market Value' column per fund and ...
newTMV=[]
for i, row in notebook.tqdm(wbH.iterrows()):
    if nav[row['Entity ID']] == 0:
        fndpct = 1
    else:
        fndpct = row['Sum of Market Value Income'] / nav[row['Entity ID']] * 100
    
    newTMV.append(fndpct)
    
# ... replace the '% of Total Market Value' with the values in the new list
wbH['% of Total Market Value'] = newTMV # wbH['% of Total Market Value'].sum(), check, should equal number of funds



# change the 'Current Exposure %" column (AI) to the fund-specific current % of NAV based on "Current Exposure" column (AH)

# recalc the '% of Total Market Value' column per fund and ...
newCEp=[] # new Current Exposure % column
for i, row in notebook.tqdm(wbH.iterrows()):
    if nav[row['Entity ID']] == 0:
        currentexposurepct = 1
    else:
        currentexposurepct = row['Current Exposure'] / nav[row['Entity ID']] * 100
    
    newCEp.append(currentexposurepct )
    
# ... replace the '% of Total Market Value' with the values in the new list
wbH['Current Exposure %'] = newCEp # wbH['% of Total Market Value'].sum(), check, should equal number of funds

print(f"--- {round(time.time() - start_time,1)} seconds ---")

0it [00:00, ?it/s]

--- 0.4 seconds ---


In [7]:
# copy the holdings and deltas data per fund to the Derv.xslx calculation sheet
start_time = time.time()

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook

for fname in notebook.tqdm(fnames):
    wbDervCalc = load_workbook(fDx)          # load the derivative calculation file as an openpyxl workbook (afresh)
    
    # populate the 'Data' sheet in Derv.calc ... 
    wsH = wbDervCalc['Data'] # the 'Data' sheet of Derv.xslx
    fundCode = wbH[wbH['Entity Name'] == fname].iloc[0, 36] # fund code to be included in name of saved file
    rows = dataframe_to_rows(wbH[wbH['Entity Name'] == fname], index = False)
    for r_idx, row in enumerate(rows, 1):
        for c_idx, value in enumerate(row, 1):
             wsH.cell(row=r_idx, column=c_idx, value=value)

    # ... then populate the 'Deltas' sheet in Derv.calc ...
    wsD = wbDervCalc['Deltas'] # the 'Deltas' sheet of Derv.xlsx
    rows = dataframe_to_rows(wbD[wbD['Entity Name'] == fname], index = False)
    for r_idx, row in enumerate(rows, 1):
        for c_idx, value in enumerate(row, 1):
             wsD.cell(row=r_idx, column=c_idx, value=value)

    # ... and then save the derivative calculation sheet
    wbDervCalc.save(os.path.join(pthSave,f'{fundCode} Derv Calc {rptDate}.xlsx')) # save the file

wbDervCalc.close() # close the laoded workbook

print(f"--- {round(time.time() - start_time,1)} seconds ---") # for 77 files stored locally, this takes around 2½ to 3 min

NameError: name 'fnames' is not defined

In [ ]:
#------------------ ========= ------------------

In [22]:
oppxl1 = wbH.iloc[0, 35]
oppxl2 = wbH.iloc[5, 35]
print(oppxl1, type(oppxl1))

2022-05-16 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [29]:
tre = r'\\pim-cpt-statpro\SPC_602\Client\Profiles\PIM\Exports\PABS Derv Calc 16May2022.xlsx'

wbC = pd.read_excel(tre, sheet_name = 'Data')

print(wbC.iloc[1,35],type(wbC.iloc[1,35]), wbC.iloc[1,15],type(wbC.iloc[1,15]))

2022-05-16 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'> 2048-02-28 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [30]:
wbH.iloc[1,35]+5

TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

In [ ]:
# TEST AN ITERATION OF THE LOOP

fname = 'PIM Balanced'

wbDervCalc = load_workbook(fDx)          # load the derivative calculation file as an openpyxl workbook afresh
# populate the 'Data' sheet in Derv.calc ... 
wsH = wbDervCalc['Data'] # the 'Data' sheet of Derv.xslx
subsetH = wbH[wbH['Entity Name'] == fname]
fundCode = subsetH.iloc[0, 36] # fund code to be included in name of saved file
rows = dataframe_to_rows(subsetH, index = False)
for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
         wsH.cell(row=r_idx, column=c_idx, value=value)
            
# ... then populate the 'Deltas' sheet in Derv.calc ...
wsD = wbDervCalc['Deltas'] # the 'Deltas' sheet of Derv.xlsx
subsetD = wbD[wbD['Entity Name'] == fname]
rows = dataframe_to_rows(subsetD, index = False)
for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
         wsD.cell(row=r_idx, column=c_idx, value=value)

# ... and then save the derivative calculation sheet
wbDervCalc.save(os.path.join(pthSave,f'{fundCode} Derv Calc {rptDate}.xlsx')) # save the file